In [0]:
# --------------------------------
# Bronze Layer - Medallion Architecture (Service Principal Auth)
# --------------------------------


# Fetch Service Principal credentials securely
client_id = dbutils.secrets.get(scope="capstonescope", key="sp-client-id")
client_secret = dbutils.secrets.get(scope="capstonescope", key="sp-client-secret")
tenant_id = dbutils.secrets.get(scope="capstonescope", key="sp-tenant-id")


# Storage account and container details
storage_account = "capstoneblobsa"
container_name = "bronze"


# Configure Spark for Service Principal + OAuth
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type","org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint",f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


# Base path for Bronze layer
abfss_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/"


# -------------------------------
# Sample Reads for Validation
# -------------------------------
df_json_sample = spark.read.text(f"{abfss_path}house-price-parquet.json")
print("JSON Sample: ")
display(df_json_sample.limit(5))


df_orders_sample = spark.read.text(f"{abfss_path}sales.orders.txt")
print("Sales Orders Sample: ")
display(df_orders_sample.limit(5))


df_world_sample = spark.read.text(f"{abfss_path}world-population.txt")
print("World Population Sample: ")
display(df_world_sample.limit(5))


# -------------------------------
# Ingest Raw Data into Bronze
# -------------------------------
df_house_bronze = spark.read.text(f"{abfss_path}house-price-parquet.json")
df_house_bronze.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"{abfss_path}house_price")


df_orders_bronze = spark.read.text(f"{abfss_path}sales.orders.txt")
df_orders_bronze.write.format("delta").mode("overwrite").save(f"{abfss_path}sales_orders")


df_world_bronze = spark.read.text(f"{abfss_path}world-population.txt")
df_world_bronze.write.format("delta").mode("overwrite").save(f"{abfss_path}world_population")


# -------------------------------
# Delta Lake Time Travel Examples
# -------------------------------
df_house_v0 = spark.read.format("delta").option("versionAsOf", 0).load(f"{abfss_path}house_price")


df_orders_old = spark.read.format("delta").option("timestampAsOf", "2025-09-27T05:00:00").load(f"{abfss_path}sales_orders")


print("House Price Data Version 0: ")
display(df_house_v0.limit(5))


print("Sales Orders Data as of 2025-09-27T05:00:00: ")
display(df_orders_old.limit(5))

JSON Sample: 


value
"{""price"":""13300000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""2"",""stories"":""3"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""furnished""}"
"{""price"":""12250000"",""area"":""8960"",""bedrooms"":""4"",""bathrooms"":""4"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""3"",""prefarea"":""no"",""furnishingstatus"":""furnished""}"
"{""price"":""12250000"",""area"":""9960"",""bedrooms"":""3"",""bathrooms"":""2"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}"
"{""price"":""12215000"",""area"":""7500"",""bedrooms"":""4"",""bathrooms"":""2"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""3"",""prefarea"":""yes"",""furnishingstatus"":""furnished""}"
"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}"


Sales Orders Sample: 


value
"order_id,order_date,customer_id,region,category,amount"
"1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25"
"2,2025-09-03 00:00:00.0000000,103,""East"",""Office Supplies"",450.75"
"3,2025-09-03 00:00:00.0000000,103,""East"",""Office Supplies"",450.75"
"4,2025-09-04 00:00:00.0000000,104,""North"",""Electronics"",9800.00"


World Population Sample: 


value
"Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage"
"""36"",""AFG"",""Afghanistan"",""Kabul"",""Asia"",""41128771"",""38972230"",""33753499"",""28189672"",""19542982"",""10694796"",""12486631"",""10752971"",""652230"",""63.0587"",""1.0257"",""0.52"""
"""138"",""ALB"",""Albania"",""Tirana"",""Europe"",""2842321"",""2866849"",""2882481"",""2913399"",""3182021"",""3295066"",""2941651"",""2324731"",""28748"",""98.8702"",""0.9957"",""0.04"""
"""34"",""DZA"",""Algeria"",""Algiers"",""Africa"",""44903225"",""43451666"",""39543154"",""35856344"",""30774621"",""25518074"",""18739378"",""13795915"",""2381741"",""18.8531"",""1.0164"",""0.56"""
"""213"",""ASM"",""American Samoa"",""Pago Pago"",""Oceania"",""44273"",""46189"",""51368"",""54849"",""58230"",""47818"",""32886"",""27075"",""199"",""222.4774"",""0.9831"",""0"""


House Price Data Version 0: 


value
"{""price"":""13300000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""2"",""stories"":""3"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""furnished""}"
"{""price"":""12250000"",""area"":""8960"",""bedrooms"":""4"",""bathrooms"":""4"",""stories"":""4"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""no"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""3"",""prefarea"":""no"",""furnishingstatus"":""furnished""}"
"{""price"":""12250000"",""area"":""9960"",""bedrooms"":""3"",""bathrooms"":""2"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""no"",""parking"":""2"",""prefarea"":""yes"",""furnishingstatus"":""semi-furnished""}"
"{""price"":""12215000"",""area"":""7500"",""bedrooms"":""4"",""bathrooms"":""2"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""no"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""3"",""prefarea"":""yes"",""furnishingstatus"":""furnished""}"
"{""price"":""11410000"",""area"":""7420"",""bedrooms"":""4"",""bathrooms"":""1"",""stories"":""2"",""mainroad"":""yes"",""guestroom"":""yes"",""basement"":""yes"",""hotwaterheating"":""no"",""airconditioning"":""yes"",""parking"":""2"",""prefarea"":""no"",""furnishingstatus"":""furnished""}"


Sales Orders Data as of 2025-09-27T05:00:00: 


value
"order_id,order_date,customer_id,region,category,amount"
"1,2025-10-20 00:00:00.0000000,150,""South"",""Furniture"",7700.25"
"2,2025-09-03 00:00:00.0000000,103,""East"",""Office Supplies"",450.75"
"3,2025-09-03 00:00:00.0000000,103,""East"",""Office Supplies"",450.75"
"4,2025-09-04 00:00:00.0000000,104,""North"",""Electronics"",9800.00"
